In [39]:
import json
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import en_core_web_md

In [40]:
nlp = en_core_web_md.load()

In [18]:
with open('../../../../../corpora/snli_1.0/snli_1.0_train.jsonl') as f:
    train_data = [json.loads(line) for line in f.readlines()]
    
with open('../../../../../corpora/snli_1.0/snli_1.0_dev.jsonl') as f:
    dev_data = [json.loads(line) for line in f.readlines()]
    
with open('../../../../../corpora/snli_1.0/snli_1.0_test.jsonl') as f:
    test_data = [json.loads(line) for line in f.readlines()]

In [298]:
def compose(*funcs):
    def inner(*arg):
        res = {}
        for f in funcs:
            res.update(f(*arg))
        return res
    return inner


def get_classifier():
    pipe = Pipeline([
        ('dict_vect', DictVectorizer()),
        ('lrc', LogisticRegression(random_state=42, multi_class='multinomial',
                                   max_iter=100, solver='sag', n_jobs=20))])

    return pipe

# FOR FUTURE IMPROVEMENTS
def get_nlp_docs(s1, s2):
    return nlp(s1), nlp(s2)


def get_intersection(ents1, ents2):
    setA = set(ents1)
    setB = set(ents2)
    universe = set(doc1) | set(doc2)

    return len(setA & setB)/(len(universe))


def feature_extractor_base(doc1, doc2):
    feats = {}
    feats['similarity'] = doc1.similarity(doc2)
    
    return feats

# It makes it a bit worse
# TODO: investigate and improve
def feature_extractor_inter_ner(doc1, doc2):
    feats = {}

    feats['ner-inter'] = get_intersection(
        [(x.label_, x.lemma_) for x in doc1.ents],
        [(x.label_, x.lemma_) for x in doc2.ents]
    )
    
    return feats


def feature_extractor_inter_word(doc1, doc2):
    feats = {}

    feats['w-inter'] = get_intersection(
            [x.lemma_ for x in doc1],
            [x.lemma_ for x in doc2]
        )
    
    return feats


def feature_extractor_inter_noun(doc1, doc2):
    feats = {}

    feats['n-inter'] = get_intersection(
            [x.lemma_ for x in doc1 if x.pos_ == 'NOUN'],
            [x.lemma_ for x in doc2 if x.pos_ == 'NOUN']
        )
    
    return feats


def feature_extractor_inter_verb(doc1, doc2):
    feats = {}

    feats['v-inter'] = get_intersection(
            [x.lemma_ for x in doc1 if x.pos_ == 'VERB'],
            [x.lemma_ for x in doc2 if x.pos_ == 'VERB']
        )
    
    return feats


def feature_extractor_inter_num(doc1, doc2):
    feats = {}

    feats['v-inter'] = get_intersection(
            [x.lemma_ for x in doc1 if x.pos_ == 'NUM'],
            [x.lemma_ for x in doc2 if x.pos_ == 'NUM']
        )
    
    return feats
    

def get_data(dataset, feature_extractor):
    features = [feature_extractor(nlp(x['sentence1']), nlp(x['sentence2'])) for x in dataset]
    labels = [x['gold_label'] for x in dataset]
    return features, labels


def print_result(train_data, dev_data, feature_extractor):
    X_train, y_train = get_data(train_data[:200], feature_extractor)
    X_dev, y_dev = get_data(dev_data[:200], feature_extractor)
    clf.fit(X_train, y_train)
    print(classification_report(y_dev, clf.predict(X_dev)))

In [29]:
clf = get_classifier()

### Baseline (just simply use sentence similarity from spacy)

In [264]:
print_result(train_data, dev_data, feature_extractor_base)

               precision    recall  f1-score   support

            -       0.00      0.00      0.00         1
contradiction       0.41      0.32      0.36        69
   entailment       0.38      0.67      0.48        66
      neutral       0.17      0.08      0.11        64

     accuracy                           0.36       200
    macro avg       0.24      0.27      0.24       200
 weighted avg       0.32      0.35      0.32       200



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 1. With NER intersection (-)

In [276]:
feature_extractor = compose(feature_extractor_base, feature_extractor_inter_ner)
print_result(train_data, dev_data, feature_extractor)

               precision    recall  f1-score   support

            -       0.00      0.00      0.00         1
contradiction       0.40      0.30      0.34        69
   entailment       0.38      0.67      0.48        66
      neutral       0.16      0.08      0.11        64

     accuracy                           0.35       200
    macro avg       0.23      0.26      0.23       200
 weighted avg       0.31      0.35      0.31       200



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2. With word intersection without NER intersection (?)

In [285]:
feature_extractor = compose(feature_extractor_base,
                            feature_extractor_inter_word)
print_result(train_data, dev_data, feature_extractor)

               precision    recall  f1-score   support

            -       0.00      0.00      0.00         1
contradiction       0.40      0.48      0.44        69
   entailment       0.37      0.59      0.46        66
      neutral       0.15      0.03      0.05        64

     accuracy                           0.37       200
    macro avg       0.23      0.28      0.24       200
 weighted avg       0.31      0.37      0.32       200



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 3. With word intersection with NER intersection (+)

In [287]:
# TODO: NER words, lemma etc
feature_extractor = compose(feature_extractor_base,
                            feature_extractor_inter_ner,
                            feature_extractor_inter_word)
print_result(train_data, dev_data, feature_extractor)

               precision    recall  f1-score   support

            -       0.00      0.00      0.00         1
contradiction       0.40      0.48      0.44        69
   entailment       0.37      0.59      0.46        66
      neutral       0.15      0.03      0.05        64

     accuracy                           0.37       200
    macro avg       0.23      0.28      0.24       200
 weighted avg       0.31      0.37      0.32       200



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 4. With NOUN intersection (+)

In [294]:
feature_extractor = compose(feature_extractor_base,
                            feature_extractor_inter_ner,
                            feature_extractor_inter_word,
                            feature_extractor_inter_noun)
print_result(train_data, dev_data, feature_extractor)

               precision    recall  f1-score   support

            -       0.00      0.00      0.00         1
contradiction       0.42      0.48      0.45        69
   entailment       0.50      0.53      0.51        66
      neutral       0.41      0.33      0.37        64

     accuracy                           0.45       200
    macro avg       0.33      0.33      0.33       200
 weighted avg       0.44      0.45      0.44       200



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [297]:
# VERB inter (-)
feature_extractor = compose(feature_extractor_base,
                            feature_extractor_inter_ner,
                            feature_extractor_inter_word,
                            feature_extractor_inter_noun,
                            feature_extractor_inter_verb)
print_result(train_data, dev_data, feature_extractor)

               precision    recall  f1-score   support

            -       0.00      0.00      0.00         1
contradiction       0.43      0.48      0.46        69
   entailment       0.47      0.58      0.52        66
      neutral       0.30      0.20      0.24        64

     accuracy                           0.42       200
    macro avg       0.30      0.31      0.30       200
 weighted avg       0.40      0.42      0.41       200



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [305]:
# NUM inter (-)
feature_extractor = compose(feature_extractor_base,
                            feature_extractor_inter_ner,
                            feature_extractor_inter_word,
                            feature_extractor_inter_noun,
                            feature_extractor_inter_verb,
                            feature_extractor_inter_num
                           )
print_result(train_data, dev_data, feature_extractor)

               precision    recall  f1-score   support

            -       0.00      0.00      0.00         1
contradiction       0.42      0.48      0.45        69
   entailment       0.50      0.53      0.51        66
      neutral       0.41      0.33      0.37        64

     accuracy                           0.45       200
    macro avg       0.33      0.33      0.33       200
 weighted avg       0.44      0.45      0.44       200



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
doc1 = nlp('Sent one, another one')
doc2 = nlp('Sent two')
set([x for x in doc1]) | set([x for x in doc2])
print(set(doc1))

{,, one, Sent, one, another}


In [166]:
def compose(*funcs):
    def inner(*arg):
        res = {}
        for f in funcs:
            print(f(*arg))
            res.update(f(*arg)[0])
        return res
    return inner

def f1(s1, s2):
    return {'d': 9}, s1, s2

def f2(s1, s2):
    return {'u': 8}, s1, s2

comp = compose(f1, f2)
comp('ww', 'gg')

({'d': 9}, 'ww', 'gg')
({'u': 8}, 'ww', 'gg')


{'d': 9, 'u': 8}

In [70]:
len(set(listA)&set(listB)) / float(len(set(listA) | set(listB))) * 100

75.0

In [215]:
a = ({}, 'fff', 'oö')
print(*(a[1], a[2]))

fff oö
